In [1]:
from pscdb_dataset import ProteinPairGraphBuilder
from tqdm import tqdm
import pandas as pd
import torch

import warnings
from Bio import BiopythonWarning
warnings.filterwarnings("ignore", category=BiopythonWarning)


In [2]:
df = pd.read_csv('data/PSCDB/structural_rearrangement_data.csv')
df.head()

,Unnamed: 0,level_0,index,PSCID,Protein Name,Free form,Bound form,Ligands,Classification(?),motion_type,Free PDB,Free Chains,Bound PDB,Bound Chains
0,0,0,0,CD.1,HYPOTHETICAL OXIDOREDUCTASE YIAK,1nxu_AB,1s20_AB,"2xNAD,2xTLA",200004,coupled_domain_motion,1nxu,AB,1s20,AB
1,1,1,1,CD.2,ADENYLATE KINASE,4ake_A,2eck_A,"ADP,AMP",200003,coupled_domain_motion,4ake,A,2eck,A
2,2,2,2,CD.3,GLUCOKINASE,1q18_AB,1sz2_AB,2xBGC,200003,coupled_domain_motion,1q18,AB,1sz2,AB
3,3,3,3,CD.4,LACTOFERRIN,1lfh_A,1lfi_A,"2xCU,2xNAG",110103,coupled_domain_motion,1lfh,A,1lfi,A
4,4,4,4,CD.5,ELONGATION FACTOR 2,1n0v_D,1n0u_A,SO1,110002,coupled_domain_motion,1n0v,D,1n0u,A


In [3]:
df.shape

(891, 14)

In [4]:
df_parsed = df[['Free PDB', 'Bound PDB', 'motion_type']]
df_parsed

,Free PDB,Bound PDB,motion_type
0,1nxu,1s20,coupled_domain_motion
1,4ake,2eck,coupled_domain_motion
2,1q18,1sz2,coupled_domain_motion
3,1lfh,1lfi,coupled_domain_motion
4,1n0v,1n0u,coupled_domain_motion
...,...,...,...
886,2dec,2df8,other_motion
887,3bu9,3bu1,other_motion
888,2f8h,2f7v,other_motion
889,2qht,2qhv,other_motion


In [5]:
# Create int labels for motion_type
df_parsed['motion_type'] = pd.Categorical(df_parsed['motion_type']).copy()
df_parsed['motion_type_int'] = df_parsed['motion_type'].cat.codes

# Mappings from int label to string label
int_to_name = dict(enumerate(df_parsed['motion_type'].cat.categories))
name_to_int = {v: k for k, v in int_to_name.items()}

df_parsed.head()


/tmp/ipykernel_10805/3995635854.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_parsed['motion_type'] = pd.Categorical(df_parsed['motion_type']).copy()
/tmp/ipykernel_10805/3995635854.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_parsed['motion_type_int'] = df_parsed['motion_type'].cat.codes


,Free PDB,Bound PDB,motion_type,motion_type_int
0,1nxu,1s20,coupled_domain_motion,1
1,4ake,2eck,coupled_domain_motion,1
2,1q18,1sz2,coupled_domain_motion,1
3,1lfh,1lfi,coupled_domain_motion,1
4,1n0v,1n0u,coupled_domain_motion,1


In [6]:
three_to_one = {
    'ALA': 'A', 'ARG': 'R', 'ASN': 'N', 'ASP': 'D',
    'CYS': 'C', 'GLN': 'Q', 'GLU': 'E', 'GLY': 'G',
    'HIS': 'H', 'ILE': 'I', 'LEU': 'L', 'LYS': 'K',
    'MET': 'M', 'PHE': 'F', 'PRO': 'P', 'SER': 'S',
    'THR': 'T', 'TRP': 'W', 'TYR': 'Y', 'VAL': 'V'
}
amino_acids = sorted(three_to_one.values())  # Sorted alphabetically
idx_to_aa = {i: aa for i, aa in enumerate(amino_acids)}
idx_to_aa

{0: 'A',
 1: 'C',
 2: 'D',
 3: 'E',
 4: 'F',
 5: 'G',
 6: 'H',
 7: 'I',
 8: 'K',
 9: 'L',
 10: 'M',
 11: 'N',
 12: 'P',
 13: 'Q',
 14: 'R',
 15: 'S',
 16: 'T',
 17: 'V',
 18: 'W',
 19: 'Y'}

In [7]:
"""import logging
import traceback
from tqdm import tqdm

# Configure logging
logging.basicConfig(
    filename='pdb_processing.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

class DebuggableBuilder(ProteinPairGraphBuilder):
    def build_graph_pair(self, free_pdb, bound_pdb, motion_label):
        try:
            logging.info(f"Processing {free_pdb} & {bound_pdb}")
            result = super().build_graph_pair(free_pdb, bound_pdb, motion_label)
            
            if not result:
                logging.warning(f"Skipped {free_pdb}-{bound_pdb}: No valid data")
                
            return result
            
        except Exception as e:
            logging.error(f"CRASHED ON {free_pdb}-{bound_pdb}:")
            logging.error(traceback.format_exc())
            print(f"\n\nCritical error processing {free_pdb}-{bound_pdb}")
            print(traceback.format_exc())
            return None

def process_with_resume(df, builder, existing_data=[]):
    my_dataset = existing_data
    
    try:
        for idx, row in tqdm(df.iterrows(), total=len(df)):
            if idx < len(my_dataset):
                continue
                
            data = builder.build_graph_pair(
                row['Free PDB'],
                row['Bound PDB'],
                row['motion_type_int']
            )
            if data:
                my_dataset.append(data)
                
            # Save progress every 50 entries
            if idx % 50 == 0:
                torch.save(my_dataset, f'data/PSCDB/graphs/partial_dataset_{idx}.pt')
                
    except KeyboardInterrupt:
        print("\nUser interrupted. Saving partial dataset...")
        return my_dataset
    except Exception as e:
        logging.error("Fatal error:")
        logging.error(traceback.format_exc())
        print("\nFatal error - partial dataset saved")
        return my_dataset
        
    return my_dataset

# Usage
builder = DebuggableBuilder(threshold=5.0, data_path='data/PSCDB/PDB_structures')

# Try to load existing partial dataset
try:
    my_dataset = torch.load('partial_dataset.pt', weights_only=False)['dataset']
    print(f"Resuming from {len(my_dataset)} existing entries")
except FileNotFoundError:
    my_dataset = []

my_dataset = process_with_resume(df_parsed, builder, my_dataset)

# Final save
torch.save(my_dataset, 'complete_dataset.pt')
print(f"Built {len(my_dataset)} valid graph pairs")"""

'import logging\nimport traceback\nfrom tqdm import tqdm\n\n# Configure logging\nlogging.basicConfig(\n    filename=\'pdb_processing.log\',\n    level=logging.INFO,\n    format=\'%(asctime)s - %(levelname)s - %(message)s\'\n)\n\nclass DebuggableBuilder(ProteinPairGraphBuilder):\n    def build_graph_pair(self, free_pdb, bound_pdb, motion_label):\n        try:\n            logging.info(f"Processing {free_pdb} & {bound_pdb}")\n            result = super().build_graph_pair(free_pdb, bound_pdb, motion_label)\n            \n            if not result:\n                logging.warning(f"Skipped {free_pdb}-{bound_pdb}: No valid data")\n                \n            return result\n            \n        except Exception as e:\n            logging.error(f"CRASHED ON {free_pdb}-{bound_pdb}:")\n            logging.error(traceback.format_exc())\n            print(f"\n\nCritical error processing {free_pdb}-{bound_pdb}")\n            print(traceback.format_exc())\n            return None\n\ndef process

In [8]:
builder = ProteinPairGraphBuilder(threshold=5.0, data_path='data/PSCDB/PDB_structures')
my_dataset = []

for _, row in tqdm(df_parsed.iterrows(), total=len(df_parsed)):
    data = builder.build_graph_pair(
        row['Free PDB'],
        row['Bound PDB'],
        row['motion_type_int']
    )
    if data:
        my_dataset.append(data)

print(f"\n\nBuilt {len(my_dataset)} valid graph pairs")

  0%|          | 0/891 [00:00<?, ?it/s]

Structure exists: 'data/PSCDB/PDB_structures/pdb1nxu.ent' 
Structure exists: 'data/PSCDB/PDB_structures/pdb1s20.ent' 


  0%|          | 2/891 [00:00<06:02,  2.45it/s]

Aligned 317 residues (Free: 317, Bound: 319, Excluded: 2)
Structure exists: 'data/PSCDB/PDB_structures/pdb4ake.ent' 
Structure exists: 'data/PSCDB/PDB_structures/pdb2eck.ent' 
Aligned 214 residues (Free: 214, Bound: 214, Excluded: 0)
Structure exists: 'data/PSCDB/PDB_structures/pdb1q18.ent' 


  0%|          | 3/891 [00:01<04:31,  3.27it/s]

...

Aligned 336 residues (Free: 336, Bound: 336, Excluded: 0)
Structure exists: 'data/PSCDB/PDB_structures/pdb2dec.ent' 
Structure exists: 'data/PSCDB/PDB_structures/pdb2df8.ent' 


100%|█████████▉| 889/891 [09:27<00:00,  5.36it/s]

Aligned 325 residues (Free: 325, Bound: 325, Excluded: 0)
Structure exists: 'data/PSCDB/PDB_structures/pdb3bu9.ent' 
Structure exists: 'data/PSCDB/PDB_structures/pdb3bu1.ent' 
Aligned 140 residues (Free: 140, Bound: 144, Excluded: 4)
Structure exists: 'data/PSCDB/PDB_structures/pdb2f8h.ent' 
Structure exists: 'data/PSCDB/PDB_structures/pdb2f7v.ent' 
Aligned 360 residues (Free: 360, Bound: 360, Excluded: 0)
Structure exists: 'data/PSCDB/PDB_structures/pdb2qht.ent' 


100%|██████████| 891/891 [09:28<00:00,  1.57it/s]

Structure exists: 'data/PSCDB/PDB_structures/pdb2qhv.ent' 
Aligned 210 residues (Free: 210, Bound: 210, Excluded: 0)
Structure exists: 'data/PSCDB/PDB_structures/pdb1uj4.ent' 
Structure exists: 'data/PSCDB/PDB_structures/pdb1uj5.ent' 
Aligned 221 residues (Free: 221, Bound: 225, Excluded: 4)


Built 856 valid graph pairs


In [9]:
print(f"One-hot Amino Acid: \n{my_dataset[0].x[0,:20]}")
print(f"Amino Acid Label: {idx_to_aa[(my_dataset[0].x[0,:20]).argmax().item()]}")
print(f"Free Structure Amino Acid Coords (x, y, z):   {my_dataset[0].x[0,20:23]}")
print(f"Bound Structure Amino Acid Coords (x, y, z):  {my_dataset[0].x[0,23:26]}")
print(f"Displacement Coords (Bound - Free) (x, y, z): {my_dataset[0].x[0,26:29]}")


One-hot Amino Acid: 
tensor([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.])
Amino Acid Label: K
Free Structure Amino Acid Coords (x, y, z):   tensor([-23.0000, -12.9500, -39.1070])
Bound Structure Amino Acid Coords (x, y, z):  tensor([89.3800, 75.6730, 79.5710])
Displacement Coords (Bound - Free) (x, y, z): tensor([112.3800,  88.6230, 118.6780])


In [10]:
for i in range(len(my_dataset)):
    if not my_dataset[i].x.__len__() == my_dataset[i].x[:,:20].sum().item():
        print(f"Missing amino acid(s) in data point{i}")

In [11]:
my_dataset[:5]

[Data(x=[317, 29], y=[1], edge_index_free=[2, 385], edge_index_bound=[2, 389], edge_index_union=[2, 408], pos_free=[317, 3], pos_bound=[317, 3]),
 Data(x=[214, 29], y=[1], edge_index_free=[2, 270], edge_index_bound=[2, 265], edge_index_union=[2, 294], pos_free=[214, 3], pos_bound=[214, 3]),
 Data(x=[314, 29], y=[1], edge_index_free=[2, 403], edge_index_bound=[2, 412], edge_index_union=[2, 427], pos_free=[314, 3], pos_bound=[314, 3]),
 Data(x=[691, 29], y=[1], edge_index_free=[2, 889], edge_index_bound=[2, 931], edge_index_union=[2, 979], pos_free=[691, 3], pos_bound=[691, 3]),
 Data(x=[429, 29], y=[1], edge_index_free=[2, 560], edge_index_bound=[2, 534], edge_index_union=[2, 608], pos_free=[429, 3], pos_bound=[429, 3])]

In [12]:
torch.save(my_dataset, 'protein_motion_dataset.pt')

In [13]:
dataset = torch.load('protein_motion_dataset.pt', weights_only=False)
len(dataset)

856